# Training Model: SVM

## 1. Importing libraries ...

In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.ml_prep import prep_ml
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

## 2. Importing Participants..

In [2]:
#participants = ["904", "905", "906", "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]
# participants = [ "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]
#9m with >40 trials
participants = [ "909", "912", "908", "913", "914", "916", "917", "919", "920", "921", "924","927", "930"]

#12m all
#participants = ["109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]
#12m with >40 trials
# participants = ["109", "111", "112", "115", "124"]


# 3 Split by ps first 

In [3]:
participants_train, participants_test = train_test_split(participants,test_size=0.2)
print(len(participants_train), len(participants_test))

X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

# X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
# X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

# X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
# X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

14 4
loaded
loaded


In [4]:
(xtrain, d) = X_train[0][0].shape
xtrain


224

In [5]:
(xtest, d) = X_test[0][0].shape
xtest

16

In [6]:
ytrain = y_train[0][0].shape[0]
ytrain



64

In [7]:
ytest = y_test[0][0].shape[0]
ytest

16

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

## 3. Set up averaging

In [6]:
# X, y, good_trial_count = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")
# Xt, yt, good_trial_count_t = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials")
# Xa, ya, good_trial_count_a = prep_ml(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants")



loaded


## Set up split 
See ml_svm avg to split ps instead (this way just splits the dataset)

Does NOT work correctly

In [7]:
## X_train, X_test, y_train, y_test = train_test_split(
##    Xt, yt, test_size=0.2, random_state=0)


4


### Karl's code

In [5]:
model = SVC(gamma='scale', kernel = 'rbf')

model.fit(X_train[0][0], y_train[0][0])
np.mean(model.predict(X_test[0][0]) != y_test[0][0])

0.46875

In [6]:

y_pred = model.predict(X_test[0][0])

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test[0][0],y_pred)))
print('Precision Score : ' + str(precision_score(y_test[0][0],y_pred)))
print('Recall Score : ' + str(recall_score(y_test[0][0],y_pred)))
print('F1 Score : ' + str(f1_score(y_test[0][0],y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test[0][0],y_pred)))

Accuracy Score : 0.53125
Precision Score : 0.5217391304347826
Recall Score : 0.75
F1 Score : 0.6153846153846153
Confusion Matrix : 
[[10 22]
 [ 8 24]]


#### from web 
https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train[0][0], y_train[0][0])

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# 